# Benchmarks

## Initialize

In [1]:
#library(Rmisc)
library(dtplyr)
library(tidyverse)
library(glue)
library(arrow)
library(patchwork)
library(data.table)
library("jsonlite")
library(ggthemes)

── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.0     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.1     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp



Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘pac

In [2]:
base_path = "/home/jakobs"

project_path = glue("{base_path}/data")
experiment = '230629'
experiment_path = glue("{project_path}/{experiment}")

endpoints_md = endpoint_defs = arrow::read_feather(glue("{base_path}/data/endpoints_epic_md.feather")) %>% filter(n_epic>=100)
endpoints = endpoint_defs$endpoint

In [3]:
today = experiment

In [24]:
base_size = 8
title_size = 10
facet_size = 10
geom_text_size=3
theme_set(theme_classic(base_size = base_size) + 
          theme(strip.background = element_blank(), plot.title=element_text(size=title_size, hjust=0), 
                strip.text.x = element_text(size = facet_size),axis.title=element_text(size=10), axis.text=element_text(size=8, color="black"),
                legend.position="bottom", axis.line = element_line(size = 0.2), axis.ticks=element_line(size=0.2), panel.grid.major=element_line()))

In [25]:
endpoint_map = endpoint_defs$phecode_string
names(endpoint_map) =  endpoint_defs$endpoint
endpoint_order = (endpoint_defs %>% arrange(as.numeric(phecode)))$endpoint

In [26]:
endpoint_selection = c(
    # generally very important
    "phecode_202", # Diabetes mellitus
    "phecode_401",	#Hypertension"  
    "phecode_404", # Ischemic heart disease   
    "phecode_404-1", # Myocardial infarction [Heart attack]
    "phecode_431-11", # Cerebral infarction [Ischemic stroke]
    "phecode_424", # Heart failure

    
    "phecode_059-1", # COVID 19
    "phecode_468", # Pneumonia
    "phecode_474", # Chronic obstructive pulmonary disease [COPD]
      
    "phecode_286-2", #	Major depressive disorder
    "phecode_324-11", #Parkinson's Disease
    "phecode_328", # Dementias and cerebral degeneration

    
    "phecode_164", # Anemia
    "phecode_726-1", # Osteoporosis
    "phecode_371", # Cataract
    "phecode_374-42", # Diabetic retinopathy
    "phecode_374-5", # Macular degeneration
    "phecode_375-1", # Glaucoma
    
    
    "phecode_103", # Malignant neoplasm of the skin
    "phecode_101", # Malignant neoplasm of the digestive organs
    "phecode_102", # LUNG CANCER
    
    "phecode_583", # Chronic kidney disease    
    "phecode_542", # Chronic liver disease and sequelae    
    "OMOP_4306655" # All-Cause Death
    
    # also generally important and relevant
    #"phecode_440-3", # Pulmonary embolism
    #"phecode_468-1",	#Viral pneumonia
#     "phecode_460-2",	#Acute lower respiratory infection
    #"phecode_388" # Blindness and low vision
      # generally important and fun to check
   # "phecode_374-3", # Retinal vascular changes and occlusions
    #"phecode_665", # Psoriasis
#     "phecode_121", # Leukemia
    # important for eye
#     "phecode_705-1", # Rheumatoid arthritis
)

endpoints_common = c(
   'phecode_164', #Anemia
 'phecode_705-1', #Rheumatoid arthritis
 'phecode_328', #Dementias and cerebral degeneration
 'phecode_328-1', #Alzheimer's disease
 'phecode_401', #Hypertension
 'phecode_202', #Diabetes mellitus
 'phecode_416-21', #Atrial fibrillation
 'phecode_404-1', #Myocardial infarction [Heart attack]
 'phecode_424', #Heart failure
 'phecode_468', #Pneumonia
 'phecode_474', #Chronic obstructive pulmonary disease [COPD]
 'phecode_583', #Chronic kidney disease
 'OMOP_4306655' #All-Cause Death
    )
    
endpoints_cardio = c(
    'phecode_438-11',   #  "Abdominal aortic aneurysm",
    'phecode_440-3',#  "Pulmonary embolism", # intervention
    'phecode_413-21',#  "Aortic stenosis", # intervention
    'phecode_400'#  "Rheumatic fever and chronic rheumatic heart diseases",	
)

endpoints_eye= c(
    'phecode_374-5', #Macular degeneration
 'phecode_374-51', #Age-related macular degeneration
 'phecode_374-42', #Diabetic retinopathy
 'phecode_371', #Cataract
 'phecode_388', #Blindness and low vision
 'phecode_367-5', #Uveitis
 'phecode_389-1' #Ocular pain
)

In [27]:
endpoint_defs = endpoint_defs %>% 
    mutate(name = phecode_string) %>%
    mutate(name = 
           case_when( 
               phecode_string == "Myocardial infarction [Heart attack]"~"Myocardial infarction",
               phecode_string == "Cerebral infarction [Ischemic stroke]"~"Ischemic stroke",
               phecode_string == "Chronic obstructive pulmonary disease [COPD]"~"Chronic obstructive pulmonary disease",
               phecode_string == "Mitral valve insufficiency"~"Mitral insufficiency",
               phecode_string == "Parkinson's disease (Primary)"~"Parkinson's disease",
               phecode_string == "Suicide ideation and attempt or self harm"~"Suicide attempt",
               phecode_string == "Ischemic heart disease"~"Coronary heart disease",
               phecode_string == "Chronic kidney disease"~"Chronic kidney disease",
               phecode_string == "Rheumatic fever and chronic rheumatic heart diseases"~"Rheumatic heart disease",
               phecode_string == "Abdominal aortic aneurysm"~"Abdominal aortic aneurysm",
                  TRUE ~ name)
           )
            
endpoint_map = endpoint_defs$name
names(endpoint_map) =  endpoint_defs$endpoint
#endpoint_order = (endpoint_defs %>% arrange(as.numeric(phecode)))$endpoint
endpoint_order = endpoint_selection

In [28]:
endpoints_cardio = c(
    'phecode_431-11', #  "Cerebral infarction [Ischemic stroke]",
    'phecode_404', #  "Ischemic heart disease",
    'phecode_404-1', #  "Myocardial infarction [Heart attack]", # intervention
    'phecode_424', #  "Heart failure", # intervention
     'OMOP_4306655', #  "All-Cause Death", # intervention
    'phecode_420' #  "Cardiac arrest", # intervention
   
)

## Load data

# Load Benchmarks

In [29]:
name = "hrs_endpoints"
hrs_endpoints = arrow::read_feather(glue("{experiment_path}/{name}.feather"), 
                                    col_select=c("endpoint", "score", "partition", "hrs_ret", "hrs_ret_age", "hrs_ret_sex")) %>% 
    left_join(endpoint_defs) %>% filter(n_epic>=100)

Joining with `by = join_by(endpoint)`


In [31]:
hrs_endpoints

endpoint,score,partition,hrs_ret,hrs_ret_age,hrs_ret_sex,n_epic,eligable,n,freq,phecode,phecode_string,phecode_category,sex,name
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>
OMOP_4306655,ASCVD+Retina,0,1.588715,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,1,1.517196,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,2,1.557983,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,3,1.542834,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,4,1.560359,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,5,1.490507,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,6,1.518358,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,7,1.560944,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death
OMOP_4306655,ASCVD+Retina,8,1.514556,NA,NA,111,61256,3548,0.05792086,4306655,All-Cause Death,Death,Both,All-Cause Death


In [32]:
hrs = hrs_endpoints %>% 
    group_by(endpoint, score, phecode_string, phecode_category) %>% 
    #filter(score %in% c("Retina", "Age+Sex+Retina", "Age+Sex+Retina+I(Age*ret)")) %>%
    #mutate(endpoint = factor(endpoint, levels=endpoints_sorted)) %>%
    #mutate(phecode_category = factor(phecode_category, levels=pcats_sorted)) %>%
    select(endpoint, score, hrs_ret, hrs_ret_age, hrs_ret_sex, phecode_string, phecode_category) %>%
    ungroup()
hrs

endpoint,score,hrs_ret,hrs_ret_age,hrs_ret_sex,phecode_string,phecode_category
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
OMOP_4306655,ASCVD+Retina,1.588715,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.517196,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.557983,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.542834,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.560359,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.490507,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.518358,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.560944,NA,NA,All-Cause Death,Death
OMOP_4306655,ASCVD+Retina,1.514556,NA,NA,All-Cause Death,Death


In [33]:
unique(hrs$score)

[1] "ASCVD+Retina"   "Age+Sex+Retina" "Retina"         "SCORE2+Retina"

In [34]:
purrr::map_dfr(integer(100), ~ hrs %>% group_by(endpoint, score) %>% sample_frac(1, replace=TRUE), .id="i") -> temp_hrs

In [35]:
library(ggdist)

In [36]:
temp_hrs_ret = temp_hrs %>% group_by(endpoint, score, phecode_string, phecode_category) %>% median_qi(hrs_ret, hrs_ret_age, hrs_ret_sex, na.rm=TRUE)

In [37]:
temp_hrs_linear = temp_hrs %>% ungroup() %>%
    filter(score %in% c("Retina",
                        "Age+Sex+Retina"
#                        "SCORE2+Retina",
#                        "ASCVD+Retina",
#                        "QRISK3+Retina"
                       )) %>%
    group_by(endpoint, score, phecode_string, phecode_category) %>% 
    median_qi(`hrs_ret`, na.rm=TRUE) %>% rename(hr = `hrs_ret`) %>%
    mutate(hr = round(hr, 2), .lower=round(.lower, 2), .upper=round(.upper, 2)) %>%#c(hr, .lower, .upper), round, 1) %>%
    mutate(HR_Retina = as.character(glue("{hr} ({.lower}, {.upper})"))) %>% 
    select(endpoint, score, phecode_string, phecode_category, HR_Retina) %>%
    pivot_wider(names_from="score", values_from="HR_Retina") %>% 
    select(all_of(c("phecode_category", "endpoint", "phecode_string", "Retina", 'Age+Sex+Retina'
#                     "SCORE2+Retina", "ASCVD+Retina","QRISK3+Retina"
                   ))) %>% 
    #mutate(endpoint = factor(endpoint, levels = endpoint_order)) %>% 
    arrange(endpoint) %>%
    #mutate(endpoint = recode(endpoint, !!!endpoint_map)) %>%
    ungroup()
    
temp_hrs_linear

phecode_category,endpoint,phecode_string,Retina,Age+Sex+Retina
<chr>,<chr>,<chr>,<chr>,<chr>
Death,OMOP_4306655,All-Cause Death,"4 (3.74, 4.38)","1.79 (1.59, 1.93)"
ID,phecode_089,Infections,"170.74 (140.14, 191.91)","9.69 (7.18, 11.6)"
ID,phecode_089-1,Bacterial infections,"6.66 (6.14, 7.34)","1.76 (1.5, 1.94)"
ID,phecode_089-2,Viral infections,"365.9 (219.33, 817.2)","103.26 (47.72, 209.17)"
ID,phecode_089-3,Fungal infections,"71.13 (31.12, 158.36)","11.65 (2.62, 39.12)"
ID,phecode_092,"Bacteremia, Sepsis, and SIRS","2.61 (2.44, 2.79)","1.38 (1.22, 1.48)"
ID,phecode_092-2,Sepsis,"2.62 (2.46, 2.81)","1.36 (1.22, 1.5)"
Neoplasms,phecode_101,Malignant neoplasm of the digestive organs,"2.28 (2.02, 2.49)","1.18 (1.02, 1.22)"
Neoplasms,phecode_101-4,Malignant neoplasm of the colon and rectum,"2.08 (1.95, 2.36)","1.12 (1.04, 1.18)"


In [49]:
ukb = fread("ST6_HRs_UKB.csv")

In [50]:
ukb

endpoint,phecode_string,Retina,Age+Sex+Retina,SCORE2+Retina,ASCVD+Retina,QRISK3+Retina
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
phecode_202,Diabetes mellitus,"3.67 (3.31, 4.03)","4.13 (3.48, 4.7)","3.47 (3.05, 3.95)","3.02 (2.71, 3.23)","2.34 (2.18, 2.55)"
phecode_401,Hypertension,"3.24 (2.92, 3.55)","2.86 (2.63, 3.2)","2.27 (2.06, 2.44)","2.05 (1.9, 2.18)","1.85 (1.72, 1.94)"
phecode_404,Ischemic heart disease,"4.06 (3.23, 4.17)","3.11 (2.52, 3.27)","2.77 (2.27, 2.88)","2.52 (2.13, 2.66)","2.31 (1.93, 2.36)"
phecode_404-1,Myocardial infarction [Heart attack],"4.52 (3.67, 4.66)","3.44 (2.9, 3.97)","3.12 (2.53, 3.41)","2.76 (2.37, 3.16)","2.5 (2.14, 2.73)"
phecode_431-11,Cerebral infarction [Ischemic stroke],"5.18 (4.3, 5.63)","4.39 (3.53, 4.59)","4.03 (3.23, 4.12)","3.88 (3.11, 3.91)","3.74 (3, 3.86)"
phecode_424,Heart failure,"4.97 (4.15, 5.4)","4.37 (3.44, 4.39)","3.88 (3.11, 3.98)","3.58 (2.96, 3.78)","3.24 (2.63, 3.33)"
phecode_059-1,COVID-19*,"5.16 (4.7, 6.48)","5.24 (4.59, 6.29)","4.8 (4.3, 5.7)","4.74 (4.19, 5.83)","4.56 (4.07, 5.5)"
phecode_468,Pneumonia,"4.91 (4.45, 5.16)","4.8 (4.14, 4.93)","4.38 (3.76, 4.49)","4.27 (3.73, 4.56)","3.95 (3.42, 4.22)"
phecode_474,Chronic obstructive pulmonary disease [COPD],"4.71 (4, 5.03)","4.67 (3.99, 5.2)","3.38 (2.96, 3.65)","3.42 (2.97, 3.71)","3.17 (2.8, 3.48)"


In [48]:
epic = temp_hrs_linear %>% select(-phecode_category, -phecode_string)
colnames(epic) = c("endpoint", "epic_unadjusted", "epic_agesex")
epic

endpoint,epic_unadjusted,epic_agesex
<chr>,<chr>,<chr>
OMOP_4306655,"4 (3.74, 4.38)","1.79 (1.59, 1.93)"
phecode_089,"170.74 (140.14, 191.91)","9.69 (7.18, 11.6)"
phecode_089-1,"6.66 (6.14, 7.34)","1.76 (1.5, 1.94)"
phecode_089-2,"365.9 (219.33, 817.2)","103.26 (47.72, 209.17)"
phecode_089-3,"71.13 (31.12, 158.36)","11.65 (2.62, 39.12)"
phecode_092,"2.61 (2.44, 2.79)","1.38 (1.22, 1.48)"
phecode_092-2,"2.62 (2.46, 2.81)","1.36 (1.22, 1.5)"
phecode_101,"2.28 (2.02, 2.49)","1.18 (1.02, 1.22)"
phecode_101-4,"2.08 (1.95, 2.36)","1.12 (1.04, 1.18)"


In [41]:
st6 = ukb %>% left_join(epic, by="endpoint") %>% as_tibble() %>% arrange(endpoint)

In [42]:
st6_clean <- st6 %>%
  mutate(across(everything(), ~ replace_na(., ""), .names = "{col}"))

In [45]:
st6_clean

endpoint,phecode_string,Retina,Age+Sex+Retina,SCORE2+Retina,ASCVD+Retina,QRISK3+Retina,epic_unadjusted,epic_agesex
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
OMOP_4306655,All-Cause Death,"4.7 (3.96, 4.93)","3.87 (3.28, 4.27)","3.43 (2.93, 3.78)","3.44 (2.95, 3.84)","3.25 (2.84, 3.74)","4 (3.74, 4.38)","1.79 (1.59, 1.93)"
phecode_059-1,COVID-19*,"5.16 (4.7, 6.48)","5.24 (4.59, 6.29)","4.8 (4.3, 5.7)","4.74 (4.19, 5.83)","4.56 (4.07, 5.5)",,
phecode_101,Malignant neoplasm of the digestive organs,"4.17 (3.66, 4.58)","3.15 (2.83, 3.47)","2.96 (2.65, 3.22)","2.91 (2.65, 3.26)","2.88 (2.6, 3.24)","2.28 (2.02, 2.49)","1.18 (1.02, 1.22)"
phecode_102,Malignant neoplasm of the thoracic and respiratory organs,"4.71 (4.25, 5.52)","4.59 (4.05, 5.62)","3.66 (3.06, 4.06)","3.69 (3.1, 4.2)","3.75 (3.24, 4.51)",,
phecode_103,Malignant neoplasm of the skin,"4.08 (3.61, 4.24)","2.15 (1.88, 2.43)","2.28 (1.91, 2.46)","1.56 (1.4, 1.7)","1.58 (1.42, 1.71)","3.57 (3.24, 3.76)","1.11 (0.99, 1.32)"
phecode_164,Anemia,"5.07 (4.84, 5.62)","5.21 (5.04, 6.1)","4.92 (4.53, 5.52)","4.3 (4.13, 5.02)","4.22 (4.1, 4.83)","2.69 (2.52, 2.84)","1.45 (1.27, 1.54)"
phecode_202,Diabetes mellitus,"3.67 (3.31, 4.03)","4.13 (3.48, 4.7)","3.47 (3.05, 3.95)","3.02 (2.71, 3.23)","2.34 (2.18, 2.55)","3.71 (3.16, 3.9)","2.99 (2.52, 3.18)"
phecode_286-2,Major depressive disorder,"5.91 (5.09, 7.17)","3.83 (3.13, 5.18)","3.53 (2.74, 4.61)","3.05 (2.51, 4.15)","2.9 (2.42, 3.7)","3.17 (2.69, 4.7)","1.99 (1.7, 3.01)"
phecode_324-11,Parkinson's disease (Primary),"5.57 (3.88, 6.13)","2.21 (1.77, 2.68)","2.15 (1.84, 2.82)","2.22 (1.8, 2.9)","2.16 (1.78, 2.83)",,


In [43]:
st6_clean %>% write_csv("outputs/SupplTable6_ukbepic_merged.csv")

In [44]:
# new, after bugfix
temp_hrs_linear %>% filter(endpoint %in% endpoint_selection) %>% select(-phecode_category)

endpoint,phecode_string,Retina,Age+Sex+Retina
<chr>,<chr>,<chr>,<chr>
OMOP_4306655,All-Cause Death,"4 (3.74, 4.38)","1.79 (1.59, 1.93)"
phecode_101,Malignant neoplasm of the digestive organs,"2.28 (2.02, 2.49)","1.18 (1.02, 1.22)"
phecode_103,Malignant neoplasm of the skin,"3.57 (3.24, 3.76)","1.11 (0.99, 1.32)"
phecode_164,Anemia,"2.69 (2.52, 2.84)","1.45 (1.27, 1.54)"
phecode_202,Diabetes mellitus,"3.71 (3.16, 3.9)","2.99 (2.52, 3.18)"
phecode_286-2,Major depressive disorder,"3.17 (2.69, 4.7)","1.99 (1.7, 3.01)"
phecode_328,Dementias and cerebral degeneration,"3.57 (3.4, 3.86)","1.14 (1.05, 1.25)"
phecode_371,Cataract,"3.7 (3.56, 3.81)","2.95 (2.84, 3.06)"
phecode_401,Hypertension,"3.86 (3.79, 4.08)","2.05 (2, 2.18)"


In [23]:
# new, after bugfix
temp_hrs_linear %>% filter(endpoint %in% endpoint_selection) %>% select(-phecode_category)

endpoint,phecode_string,Retina,Age+Sex+Retina
<chr>,<chr>,<chr>,<chr>
OMOP_4306655,All-Cause Death,"3.92 (3.74, 4.38)","1.79 (1.59, 1.93)"
phecode_101,Malignant neoplasm of the digestive organs,"2.27 (2.02, 2.49)","1.17 (1.02, 1.22)"
phecode_103,Malignant neoplasm of the skin,"3.57 (3.24, 3.76)","1.11 (0.99, 1.32)"
phecode_164,Anemia,"2.69 (2.52, 2.84)","1.42 (1.27, 1.54)"
phecode_202,Diabetes mellitus,"3.71 (3.16, 3.9)","3.04 (2.52, 3.18)"
phecode_286-2,Major depressive disorder,"3.17 (2.69, 4.7)","2.44 (1.7, 3.01)"
phecode_328,Dementias and cerebral degeneration,"3.58 (3.4, 3.86)","1.12 (1.05, 1.25)"
phecode_371,Cataract,"3.68 (3.56, 3.81)","2.95 (2.84, 3.06)"
phecode_401,Hypertension,"3.86 (3.79, 4.08)","2.06 (2, 2.18)"


In [44]:
temp_hrs_linear %>% 
    filter(phecode_string %in% tmp_sign$phecode_string) %>%
    select(-phecode_category) %>%
    write_csv("outputs/SupplTable2_HRsLinearSIGNIFICANT_CropRatio-0.66.csv")